In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

In [3]:
a = []
for i in range(14,32):
    try:
        for j in range(100):
            with open("../TrainingData/TeslaTrainingData_2019-10-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                try:
                    a.append(str(file.read()))
                except Exception as e:
                    print(e)
                    print(file.name)
    except:
        continue
for i in range(1,20):
    try:
        for j in range(100):
            with open("../TrainingData/TeslaTrainingData_2019-11-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                try:
                    a.append(str(file.read()))
                except Exception as e:
                    print(e)
    except:
        continue
print(len(a))

2700


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers
x_train = np.array(a, dtype=np.str)
x_train = x_train.astype(str)
print(x_train.shape)
y_train = np.zeros(2700)+0.314159
print(y_train.shape)
import tensorflow as tf
embed_size = 300     # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
model1 = tf.keras.Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen))
# model1.add(Bidirectional(LSTM(128, return_sequences=True)))
# model1.add(GlobalMaxPool1D())
# model1.add(Dropout(0.2))
# model1.add(Dense(64, activation='relu'))
# model1.add(Dropout(0.2))
# model1.add(Dense(32, activation='relu'))
# model1.add(Dropout(0.2))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(100), metrics=['accuracy'])
model1.summary()
x_trainR = np.array(x_train)[:2200]
y_trainR = np.array(y_train)[:2200]
x_test = np.array(x_train)[2200:]
y_test = np.array(y_train)[2200:]
x_train = x_trainR
y_train = y_trainR
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

(2700,)
(2700,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          15000000  
_________________________________________________________________
dense (Dense)                (None, 100, 1)            301       
Total params: 15,000,301
Trainable params: 15,000,301
Non-trainable params: 0
_________________________________________________________________


In [26]:
model1.fit(x_train, y_train, epochs=5, batch_size=220, validation_data=(x_test,y_test))

Train on 2200 samples, validate on 500 samples
Epoch 1/5
2200/2200 [==============================] - 2s 907us/sample - loss: 0.3193 - accuracy: 0.0000e+00 - val_loss: 0.3341 - val_accuracy: 0.0000e+00
Epoch 2/5
2200/2200 [==============================] - 2s 832us/sample - loss: 0.3343 - accuracy: 0.0000e+00 - val_loss: 0.3341 - val_accuracy: 0.0000e+00
Epoch 3/5
2200/2200 [==============================] - 2s 815us/sample - loss: 0.3343 - accuracy: 0.0000e+00 - val_loss: 0.3341 - val_accuracy: 0.0000e+00
Epoch 4/5
2200/2200 [==============================] - 2s 853us/sample - loss: 0.3343 - accuracy: 0.0000e+00 - val_loss: 0.3341 - val_accuracy: 0.0000e+00
Epoch 5/5
2200/2200 [==============================] - 2s 842us/sample - loss: 0.3343 - accuracy: 0.0000e+00 - val_loss: 0.3341 - val_accuracy: 0.0000e+00


In [ ]:
def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

In [ ]:
def sample_predict(sentence, pad):
    encoded_sample_pred_text = encoder.encode(sample_pred_text)

    if pad:
        encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = model1.predict(tf.expand_dims(encoded_sample_pred_text, 0))

    return (predictions)

In [10]:
print(model1.weights)

[<tf.Variable 'embedding_3/embeddings:0' shape=(50000, 300) dtype=float32, numpy=
array([[ 3.0912701e-02, -4.9593560e-03,  4.7769498e-02, ...,
         3.3881117e-02, -1.8121220e-02,  3.1104807e-02],
       [-5.9883809e+00,  5.9588203e+00, -5.2675862e+00, ...,
        -5.9337935e+00,  5.8765602e+00, -6.0302949e+00],
       [-4.3353953e+00,  5.9822183e+00,  5.9377379e+00, ...,
        -5.9468856e+00,  5.9439125e+00, -6.0289674e+00],
       ...,
       [ 1.0547243e-02, -2.5794411e-02, -4.8193559e-03, ...,
         6.1958805e-03, -2.4530161e-02,  2.4477128e-02],
       [-4.2608131e-02, -6.2674060e-03, -8.6550117e-03, ...,
         4.8036385e-02,  4.5751706e-03, -1.1925053e-02],
       [ 4.9055029e-02, -3.3342648e-02,  8.9760050e-03, ...,
         1.8299889e-02, -5.1050894e-03, -2.7315808e-02]], dtype=float32)>, <tf.Variable 'bidirectional_3/forward_lstm_3/kernel:0' shape=(300, 512) dtype=float32, numpy=
array([[ 2.5315142 ,  0.6627123 ,  3.336858  , ...,  2.4928002 ,
        -1.1379876 , 

In [11]:
def predict(x):
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(x))
    x =pad_sequences(tokenizer.texts_to_sequences(x), maxlen=maxlen)
    return model1.predict(x)

In [27]:
# JUNK NEURAL NETWORK

In [5]:
x_train = np.random.random(10000)*20-10
x_train = np.sort(x_train)
y_train = np.cos(x_train)
x_train

array([-9.99569923, -9.9946905 , -9.99459666, ...,  9.99647872,
        9.99664191,  9.99740685])

In [9]:
model2 = tf.keras.Sequential()
# model1.add(Embedding(max_features, embed_size, input_length=maxlen))
# model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model2.add(Dense(64, activation='relu', input_shape=(1,)))
# model2.add(Dropout(0.2))
# model2.add(Dense(32, activation='relu'))
# model1.add(Dropout(0.2))
model2.add(Dense(1))
model2.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(1e), metrics=['accuracy'])
model2.summary()
x_trainR = np.array(x_train)[:8000]
y_trainR = np.array(y_train)[:8000]
x_test = np.array(x_train)[8000:]
y_test = np.array(y_train)[8000:]
x_train = x_trainR
y_train = y_trainR

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                128       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [10]:
model2.fit(x_train, y_train, epochs=1000)

Train on 8000 samples
Epoch 1/1000
8000/8000 [==============================] - 0s 51us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 2/1000
8000/8000 [==============================] - 0s 23us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 3/1000
8000/8000 [==============================] - 0s 22us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 4/1000
8000/8000 [==============================] - 0s 22us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 5/1000
8000/8000 [==============================] - 0s 21us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 6/1000
8000/8000 [==============================] - 0s 22us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 7/1000
8000/8000 [==============================] - 0s 21us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 8/1000
8000/8000 [==============================] - 0s 21us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 9/1000
8000/8000 [==============================] - 0s 22us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 

KeyboardInterrupt: 